<a href="https://colab.research.google.com/github/raisaurabh44/Practice_001/blob/main/Project_Loan_predition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading Libraries

In [11]:
import os # path to file
import numpy as np # linear algebra
import pandas as pd # data processing
import warnings # warning filter

# Pllot libraries
import matplotlib.pyplot as plt
import seaborn as sns

# relevent ML libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# ML models
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

#defaut theam
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)


#Warning handle
warnings.filterwarnings('ignore')

ERROR: Could not find a version that satisfies the requirement warning (from versions: none)
ERROR: No matching distribution found for warning


1Cleaning
Null valus / dropna / fill na
duplicates drop
check data types for integer or object

2 outliers
drop or caping ( IQR and vincerisation )

3 EDA / Inferintial statistics (Linear regression , featire selection , anove )

Grouping/Filtering/Visulization





